# House price prediction with scikit-learn

TODO:

PCA/feature selection

Removing outliers

Streamlining analysis with scikit

Adding also tensorflow/keras


In [ ]:
import os
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

print("os.environ.get('KAGGLE_URL_BASE') == ", os.environ.get('KAGGLE_URL_BASE'))
if 'kaggle' in os.environ.get('KAGGLE_URL_BASE','localhost'):
    print("We are running on a Kaggle Server")
    IN_KAGGLE = True
else:
    IN_KAGGLE = False

In [ ]:
if IN_COLAB:
    from google.colab import files
    !pip install kaggle
    if not os.path.exists("/root/.kaggle/kaggle.json"):
        files.upload()
        !cp kaggle.json ~/.kaggle/
        !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c house-prices-advanced-regression-techniques
    if not os.path.exists("/content/train.csv"):
        !unzip /content/house-prices-advanced-regression-techniques.zip


In [ ]:
if(IN_KAGGLE):
    train_path='../input/house-prices-advanced-regression-techniques/train.csv'
    test_path='../input/house-prices-advanced-regression-techniques/test.csv'
elif(IN_COLAB):
    train_path="/content/train.csv"
    test_path="/content/test.csv"
else:
    train_path='data/train.csv'
    test_path='data/test.csv'

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
import sys
import seaborn as sns

In [ ]:
def plot_categorical(grouped,label,figsize=(12,8)):
    plt.figure(figsize=figsize)
    labels=[]
    data=[]
    for name,group in grouped:
        labels.append(name)
        data.append(group[label])
    plt.boxplot(data,labels=labels)
    plt.xlabel(name)
    plt.ylabel(label)
    plt.show()

Feature engineering function:

In [ ]:
try:
    from house_prices_script import feat_eng
except:
    def feat_eng(df):
        ''' performs dataframe cleaning and basic feature engineering'''
        #total floors
        df['TotFlrSF']=df['1stFlrSF']+df['2ndFlrSF']
        #total number of floors (1 or 2)
        df['nFlrs']=df['2ndFlrSF'].map(lambda x: int(x>0)+1.)#if df['2ndFlrSF']>0 then df['nFlrs']=2
        df['GarageType']=df['GarageType'].fillna('NoGarage')
        df['GarageQual']=df['GarageQual'].fillna('NoGarage')
        df['GarageCond']=df['GarageCond'].fillna('NoGarage')
        df['PoolQC']=df['PoolQC'].fillna('NoPool')
        df['FireplaceQu']=df['FireplaceQu'].fillna('NoFp')
        #garage cars vs house size
        #number of bathrooms vs house size/n bedrooms
        return df

# Real estate price prediction

https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques

Notes: lots of columns -> possible PCA? (see ml-basic-> clustering tutorial for info on PCA with sklearn)

In [ ]:
train,test=pd.read_csv(train_path),pd.read_csv(test_path)
train

In [ ]:
test

In [ ]:
train,test=feat_eng(train),feat_eng(test)

In [ ]:
count=test.isna().sum()
count=pd.DataFrame(count)
count[count[0]!=0]

In [ ]:
features=test.keys()


In [ ]:
len(features)

In [ ]:
label='SalePrice'

#### Plotting raw features

In [ ]:
plot=True


In [ ]:
train.hist(bins=30, figsize=(15,15))
plt.show()

In [ ]:
corrmat = train.corr(numeric_only=True)
f, ax = plt.subplots(figsize=(16, 16))
sns.heatmap(corrmat, vmax=.8, square=True,cmap="coolwarm")
plt.show()

In [ ]:
def plot_synth_f(df:pd.DataFrame,plot_type='s',sf='synth',label='SalePrice'):
    #definition of synthetic feature here
    df[sf]=df['TotFlrSF']#how do I pass a function on the dataframe?
    if plot_type=='c':
        grouped=df.groupby(sf)
        plot_categorical(grouped)
    else:
        plt.scatter(df[sf],df[label])
        plt.xlabel(sf)
        plt.ylabel(label)
        z = np.polyfit(df[sf],df[label], 1)
        p = np.poly1d(z)
        plt.plot(df[sf],p(df[sf]), color='magenta')

Let's see if we can do some feature engineering.

First: find if there are nan entries:

In [ ]:
subset_df = train.loc[:, train.isnull().any()]

subset_df

In [ ]:
def count_na_per_col(df):
    count=df.isna().sum()
    count=pd.DataFrame(count).T
    return count

In [ ]:
count_na_per_col(train)

In [ ]:
pd.concat([count_na_per_col(train),count_na_per_col(test)])

#### Selecting the interesting features

In [ ]:
int_cat_f=['SaleCondition','TotRmsAbvGrd','ExterQual','Exterior1st','Neighborhood',
            'GarageFinish','KitchenQual','SaleType','PoolQC', 'OverallCond',
            'FullBath','HouseStyle','Condition1','MSZoning','BldgType','BsmtQual']
int_num_f=['LotArea','LotFrontage','BsmtFinSF1','TotalBsmtSF',
            'GrLivArea','GarageYrBlt','GarageArea','YearBuilt','MSSubClass','GarageCars','OverallQual']
int_ord_f=[]

In [ ]:
train

In [ ]:
train.columns

In [ ]:
corrmat = train[int_num_f+[label]].corr()
f, ax = plt.subplots(figsize=(8,8))
sns.heatmap(corrmat, vmax=.8, square=True,cmap="coolwarm")
plt.show()

## Beginning regressions

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, KBinsDiscretizer, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import set_config
from sklearn.metrics import r2_score,make_scorer,mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostRegressor
#from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
train[train['GarageCars'].isna()]

In [ ]:
#try using keras model in scikit pipeline.. or just do it all in tensorflow/keras?
#from keras.wrappers.scikit_learn import KerasRegressor


In [ ]:
params = [{'regressor': [GradientBoostingRegressor()],
 'regressor__learning_rate': [0.03,0.1, 0.5, 1.0],
 'regressor__n_estimators' : [25,50, 100, 150,200]
 },
 {
    'regressor':[RandomForestRegressor()],
    'regressor__n_estimators' : [25,50, 100, 150,200]
 },
 {
   'regressor':[AdaBoostRegressor()],
   'regressor__learning_rate':[0.03,0.1, 0.5, 1.0],
   'regressor__n_estimators' : [25,50, 100, 150,200],
   'regressor__loss':['linear','square']
 }
 ,
 {
  'regressor':[LGBMRegressor()],
  'regressor__learning_rate':[0.02,0.1,0.3],
  'regressor__reg_alpha':[0,0.1,0.2,0.3,0.4],
  'regressor__n_estimators' : [25,50, 100, 150,200]
 }
 ]
params2= [{
    'regressor':[LGBMRegressor()],
    'regressor__learning_rate':[0.02,0.1,0.3],
    'regressor__reg_alpha':[0,0.1,0.2,0.3,0.4],
    'regressor__n_estimators' : [25,50, 100, 150,200]
    }]

https://stackoverflow.com/questions/38555650/try-multiple-estimator-in-one-grid-search

In [ ]:
# Define preprocessing for numeric columns (normalize them so they're on the same scale)
numeric_transformer = Pipeline(steps=[
    ('inp',SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Define preprocessing for categorical features
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

binned_transformer = Pipeline(steps=[('inp',SimpleImputer(strategy='median')),
                                    ('KBinsDiscretizer',KBinsDiscretizer(n_bins=10))])

ordinal_transformer = Pipeline(steps=[
    ('ordenc', OrdinalEncoder())])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, int_num_f),
        ('cat', categorical_transformer, int_cat_f),
        ('ord', ordinal_transformer, int_ord_f)
        ])

# Create preprocessing and training pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', GradientBoostingRegressor())])

In [ ]:

def pipeline_builder(n_cols,c_cols,o_cols,b_cols):
    numeric_transformer = Pipeline(steps=[
        ('inp',SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    binned_transformer = Pipeline(steps=[('inp',SimpleImputer(strategy='median')),
                                    ('KBinsDiscretizer',KBinsDiscretizer(n_bins=10))])
    ordinal_transformer = Pipeline(steps=[('onehot', OrdinalEncoder())])
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, n_cols),
        ('cat', categorical_transformer, c_cols),
        ('ord', ordinal_transformer, o_cols),
        ('bin',binned_transformer, b_cols)
        ])
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor())])
    pipeline
    return pipeline

In [ ]:
score = make_scorer(r2_score)

pipe=pipeline_builder(int_num_f,int_cat_f,int_ord_f,[])

gridsearch=GridSearchCV(verbose=1,estimator=pipe,param_grid=params2,scoring=score,return_train_score=False,cv=3,n_jobs=2)


Create feature an label sets, drop outlier coumns

In [ ]:
def xy_split(df,label):
    x,y=df.drop(label,axis=1),df[label]
    return x,y

In [ ]:
x,y=train.drop([523,1298]).drop(label,axis=1),train.drop([523,1298])[label]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)


In [ ]:
set_config(display='diagram')
gridsearch

In [ ]:
model1=gridsearch.fit(x_train,(y_train))

In [ ]:
model1.best_params_

In [ ]:
model1.best_score_

In [ ]:
model1.best_estimator_

In [ ]:
best_model=model1.best_estimator_
best_model=best_model.fit(x_train,(y_train))

In [ ]:
import datetime

In [ ]:
savemodel=True

In [ ]:
if savemodel:
    timestamp=datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    models_folder= 'models'
    os.makedirs(models_folder,exist_ok=True)
    model_name= 'model_'+ timestamp +'.joblib'
    modelpath= os.path.join(models_folder,model_name)
    from joblib import dump, load
    dump(model1, modelpath)
    print('Saved ' , modelpath)

In [ ]:
def score_and_visualize(model,x_test,y_test):
    predictions=model.predict(x_test)
    plt.figure(figsize=(10,10))
    plt.scatter(y_test,predictions)
    plt.xlabel('True')
    plt.ylabel('Predicted')
    msle=mean_squared_log_error(y_true=y_test, y_pred=predictions)#msle is the metric evaluated in the kaggle challange
    r2=r2_score(y_true=y_test, y_pred=predictions)
    print('r2 coefficient: ',r2,'\nmsle = ', msle)
    z = np.polyfit(y_test, predictions, 1)
    p = np.poly1d(z)
    plt.plot(y_test,p(y_test), color='magenta')
    plt.axline((0,0),slope=1,ls='--')
    plt.show()

In [ ]:
#len(model2[1].feature_importances_)

In [ ]:
score_and_visualize(model=model1,x_test=x_test,y_test=y_test)
#score_and_visualize(model=model2,x_test=x_test,y_test=y_test)

## Now make predictions on the test dataset

In [ ]:
if IN_KAGGLE:
    sample_sub=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
elif IN_COLAB:
    sample_sub=pd.read_csv('/content/sample_submission.csv')
else:
    sample_sub=pd.read_csv('data/sample_submission.csv')
sample_sub


### Make prediction and save

In [ ]:
test

In [ ]:
#make prediction on the test set and save
pred=model1.predict(test)
pred

In [ ]:
write=True

In [ ]:
if IN_KAGGLE:
    out_df=pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
elif IN_COLAB:
    out_df=pd.read_csv('/content/sample_submission.csv')
else:
    out_df=pd.read_csv('data/sample_submission.csv')
out_df['SalePrice']=pred
if write:
    out_df.to_csv('prediction.csv',index=False)

In [ ]:
f = lambda x: model1.predict(
    pd.DataFrame(x, columns=test.columns)
)

In [ ]:
test.iloc[:4,:]

In [ ]:
if IN_COLAB:
    !pip install shap

In [ ]:

import shap

# train an XGBoost model
subs=test.iloc[:100,:]

# explain the model's predictions using SHAP
# (same syntax works for LightGBM, CatBoost, scikit-learn, transformers, Spark, etc.)
explainer = shap.KernelExplainer(f, subs)
shap_values = explainer(subs)

# visualize the first prediction's explanation
shap.plots.waterfall(shap_values[0])

In [ ]:
shap.initjs()


In [ ]:
shap.initjs()
shap.plots.force(shap_values[1])

In [ ]:
shap.plots.force(shap_values[:500])

In [ ]:
shap_values

In [ ]:
shap.plots.beeswarm(shap_values)